In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import zfit
from zfit import z
from scipy.stats import chi2
from tqdm.autonotebook import tqdm
tqdm.pandas()
import multiprocess as mp

os.environ['ZFIT_DISABLE_TF_WARNINGS'] = '1'

# data = np.load("../Data/toy_dataset.csv.npy")
data = np.load("./Data/toy_dataset_smaller_peak.npy")

In [11]:
obs = zfit.Space("x", limits=(np.min(data)-1, np.max(data)+1))
zfit_data = zfit.Data.from_numpy(obs=obs, array=data[:, 0])


n_real = len(data)

mass_hypotheses = np.arange(110, 155, 5)  # 110 to 150 GeV in 5 GeV steps

n_pseudo = 200

def perform_pseudo_experiment_b(mass, obs, n_real, zfit_data):

    # SB model
    mean = zfit.Parameter(f"mean_{mass}", mass, floating=False)  # Fixed mass
    sigma = zfit.Parameter("sigma", 2.0, floating=False)  # Fixed detector resolution
    signal_model = zfit.pdf.Gauss(mu=mean, sigma=sigma, obs=obs)
    lambda_sb = zfit.Parameter(f"lambda_sb_{mass}", -0.01, -1.0, 1.0)
    exp_bkg_model_sb = zfit.pdf.Exponential(lam=lambda_sb, obs=obs)
    frac_signal = zfit.Parameter(f"frac_signal_{mass}", 0.5, 0.0, 1.0)
    sb_model = zfit.pdf.SumPDF([exp_bkg_model_sb, signal_model], fracs=frac_signal)

    # Background-only model
    lambda_bkg = zfit.Parameter("lambda_bkg", -0.01, -1.0, 1.0)
    exp_bkg_model = zfit.pdf.Exponential(lam=lambda_bkg, obs=obs)


    pseudo_data = exp_bkg_model.sample(n=n_real).numpy()
    zfit_pseudo_data = zfit.Data.from_numpy(obs=obs, array=pseudo_data[:, 0])


    ### Fit the models
    ## Background
    nll_bkg = zfit.loss.UnbinnedNLL(model=exp_bkg_model, data=zfit_pseudo_data)
    minimizer_bkg = zfit.minimize.Minuit()
    res_bkg = minimizer_bkg.minimize(nll_bkg)
    nll_bkg_min = nll_bkg.value().numpy()

    ## Signal+Background
    nll_sb = zfit.loss.UnbinnedNLL(model=sb_model, data=zfit_pseudo_data)
    minimizer_sb = zfit.minimize.Minuit()
    res_sb = minimizer_sb.minimize(nll_sb)
    nll_sb_min = nll_sb.value().numpy()

    t_stat = 2 * (nll_bkg_min - nll_sb_min)
    return t_stat


In [ ]:
results = {"mass": [], "test_statistic": []}
for mass in range(110, 151, 5):
    print(f"mass = {mass}")
    test_statistic = []
    for _ in tqdm(range(n_pseudo)):
        test_statistic += [perform_pseudo_experiment_b(mass, obs, n_real, zfit_data)]
    test_statistic = np.array(test_statistic)
    results["mass"] += [mass]*n_pseudo
    results["test_statistic"] += test_statistic.tolist()

mass = 110


  0%|          | 0/200 [00:00<?, ?it/s]

mass = 115


  0%|          | 0/200 [00:00<?, ?it/s]

mass = 120


  0%|          | 0/200 [00:00<?, ?it/s]

In [10]:
df_b = pd.DataFrame(results)
df_b.to_csv("df_b.csv", index=False)